<a href="https://colab.research.google.com/github/RtjShreyD/Mandrin2Eng_native_trans/blob/master/Training_crude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install allennlp

     |████████████████████████████████| 7.6MB 2.9MB/s 
     |████████████████████████████████| 256kB 44.1MB/s 
     |████████████████████████████████| 194kB 31.5MB/s 
     |████████████████████████████████| 245kB 46.0MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 163kB 41.0MB/s 
     |████████████████████████████████| 133kB 38.4MB/s 
     |████████████████████████████████| 1.0MB 45.5MB/s 
  Created wheel for jsonnet: filename=jsonnet-0.14.0-cp36-cp36m-linux_x86_64.whl size=3320350 sha256=60e68a2421fab7a56a21b303a0807f2ced8a6fb8be4e1aa6ee9bcb9da13071d3
  Stored in directory: /root/.cache/pip/wheels/5b/b7/83/985f0f758fbb34f14989a0fab86d18890d1cc5ae12f26967bc
  Created wheel for parsimonious: filename=parsimonious-0.8.1-cp36-none-any.whl size=42709 sha256=7e7b9e1aac68da96301606e9142f8f760303342fe5db3858ac2854b0512e0d17
  Stored in directory: /root/.cache/pip/wheels/b7/8d/e7/a0e74217d

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import itertools
import torch
import torch.optim as optim
from allennlp.data.dataset_readers.seq2seq import Seq2SeqDatasetReader
from allennlp.data.iterators import BucketIterator
from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data.tokenizers.character_tokenizer import CharacterTokenizer
from allennlp.data.tokenizers.word_tokenizer import WordTokenizer
from allennlp.data.vocabulary import Vocabulary
from allennlp.nn.activations import Activation
from allennlp.models.encoder_decoders.simple_seq2seq import SimpleSeq2Seq
from allennlp.modules.attention import LinearAttention, BilinearAttention, DotProductAttention
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper, StackedSelfAttentionEncoder
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.predictors import SimpleSeq2SeqPredictor
from allennlp.training.trainer import Trainer

In [0]:
EN_EMBEDDING_DIM = 256
ZH_EMBEDDING_DIM = 256
HIDDEN_DIM = 256
CUDA_DEVICE = -1

def main():
    reader = Seq2SeqDatasetReader(
        source_tokenizer=WordTokenizer(),
        target_tokenizer=CharacterTokenizer(),
        source_token_indexers={'tokens': SingleIdTokenIndexer()},
        target_token_indexers={'tokens': SingleIdTokenIndexer(namespace='target_tokens')})
    train_dataset = reader.read('/content/drive/My Drive/realworldnlp-master/data/mt/tatoeba.eng_cmn.train.tsv')
    validation_dataset = reader.read('/content/drive/My Drive/realworldnlp-master/data/mt/tatoeba.eng_cmn.dev.tsv')

    vocab = Vocabulary.from_instances(train_dataset + validation_dataset,
                                      min_count={'tokens': 3, 'target_tokens': 3})

    en_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                             embedding_dim=EN_EMBEDDING_DIM)
    # encoder = PytorchSeq2SeqWrapper(
    #     torch.nn.LSTM(EN_EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))
    encoder = StackedSelfAttentionEncoder(input_dim=EN_EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, projection_dim=128, feedforward_hidden_dim=128, num_layers=1, num_attention_heads=8)

    source_embedder = BasicTextFieldEmbedder({"tokens": en_embedding})

    # attention = LinearAttention(HIDDEN_DIM, HIDDEN_DIM, activation=Activation.by_name('tanh')())
    # attention = BilinearAttention(HIDDEN_DIM, HIDDEN_DIM)
    attention = DotProductAttention()

    max_decoding_steps = 20   # TODO: make this variable
    model = SimpleSeq2Seq(vocab, source_embedder, encoder, max_decoding_steps,
                          target_embedding_dim=ZH_EMBEDDING_DIM,
                          target_namespace='target_tokens',
                          attention=attention,
                          beam_size=8,
                          use_bleu=True)
    optimizer = optim.Adam(model.parameters())
    iterator = BucketIterator(batch_size=32, sorting_keys=[("source_tokens", "num_tokens")])

    iterator.index_with(vocab)

    trainer = Trainer(model=model,
                      optimizer=optimizer,
                      iterator=iterator,
                      train_dataset=train_dataset,
                      validation_dataset=validation_dataset,
                      num_epochs=1,
                      cuda_device=CUDA_DEVICE)

    for i in range(50):
        print('Epoch: {}'.format(i))
        trainer.train()

        predictor = SimpleSeq2SeqPredictor(model, reader)

        for instance in itertools.islice(validation_dataset, 10):
            print('SOURCE:', instance.fields['source_tokens'].tokens)
            print('GOLD:', instance.fields['target_tokens'].tokens)
            print('PRED:', predictor.predict_instance(instance)['predicted_tokens'])


if __name__ == '__main__':
    main()



0it [00:00, ?it/s]

203it [00:00, 2029.20it/s]

432it [00:00, 2099.88it/s]

683it [00:00, 2205.62it/s]

928it [00:00, 2273.62it/s]

1174it [00:00, 2325.53it/s]

1419it [00:00, 2361.03it/s]

1659it [00:00, 2371.99it/s]

1900it [00:00, 2382.73it/s]

2145it [00:00, 2401.09it/s]

2378it [00:01, 2362.62it/s]

2625it [00:01, 2393.71it/s]

2867it [00:01, 2399.63it/s]

3107it [00:01, 2397.26it/s]

3357it [00:01, 2425.47it/s]

3599it [00:01, 2405.78it/s]

3849it [00:01, 2432.08it/s]

4097it [00:01, 2445.01it/s]

4346it [00:01, 2456.10it/s]

4592it [00:01, 2455.84it/s]

4838it [00:02, 2422.22it/s]

5081it [00:02, 2372.09it/s]

5331it [00:02, 2407.26it/s]

5579it [00:02, 2425.78it/s]

5824it [00:02, 2432.97it/s]

6068it [00:02, 2391.42it/s]

6310it [00:02, 2397.29it/s]

6550it [00:02, 2396.29it/s]

6790it [00:02, 2392.45it/s]

7040it [00:02, 2421.76it/s]

7283it [00:03, 2365.16it/s]

7520it [00:03, 2345.28it/s]

7765it [00:03, 2374.44it/s]

8003it [00:03, 2347.74it/s]

8239it [00:03, 2350.45it/

Epoch: 0



loss: 8.0947 ||:   0%|          | 1/1169 [00:02<52:30,  2.70s/it]
loss: 8.0080 ||:   0%|          | 2/1169 [00:03<38:46,  1.99s/it]
loss: 7.9239 ||:   0%|          | 3/1169 [00:03<28:58,  1.49s/it]
loss: 7.7907 ||:   0%|          | 4/1169 [00:03<21:39,  1.12s/it]
loss: 7.7194 ||:   0%|          | 5/1169 [00:04<17:31,  1.11it/s]
loss: 7.6508 ||:   1%|          | 6/1169 [00:04<15:00,  1.29it/s]
loss: 7.5576 ||:   1%|          | 7/1169 [00:04<12:56,  1.50it/s]
loss: 7.4561 ||:   1%|          | 8/1169 [00:05<10:48,  1.79it/s]
loss: 7.3559 ||:   1%|          | 9/1169 [00:05<09:20,  2.07it/s]
loss: 7.2551 ||:   1%|          | 10/1169 [00:05<08:27,  2.29it/s]
loss: 7.1624 ||:   1%|          | 11/1169 [00:06<07:50,  2.46it/s]
loss: 7.0738 ||:   1%|          | 12/1169 [00:06<07:24,  2.61it/s]
loss: 7.0129 ||:   1%|          | 13/1169 [00:06<07:19,  2.63it/s]
loss: 6.9392 ||:   1%|          | 14/1169 [00:07<07:25,  2.59it/s]
loss: 6.8780 ||:   1%|▏         | 15/1169 [00:07<07:19,  2.62it/s]
los

SOURCE: [@start@, I, have, to, go, to, sleep, ., @end@]
GOLD: [@start@, 我, 该, 去, 睡, 觉, 了, 。, @end@]
PRED: ['我', '不', '知', '道', '他', '。']
SOURCE: [@start@, I, just, do, n't, know, what, to, say, ., @end@]
GOLD: [@start@, 我, 就, 是, 不, 知, 道, 說, 些, 什, 麼, 。, @end@]
PRED: ['我', '不', '知', '道', '他', '不', '會', '做', '。']
SOURCE: [@start@, I, may, give, up, soon, and, just, nap, instead, ., @end@]
GOLD: [@start@, 也, 许, 我, 会, 马, 上, 放, 弃, 然, 后, 去, 睡, 一, 觉, 。, @end@]
PRED: ['我', '不', '知', '道', '他', '不', '知', '道', '。']
SOURCE: [@start@, I, 'm, going, to, go, ., @end@]
GOLD: [@start@, 我, 要, 走, 了, 。, @end@]
PRED: ['我', '不', '知', '道', '。']
SOURCE: [@start@, That, 's, MY, line, !, @end@]
GOLD: [@start@, 那, 是, 我, 的, 台, 词, ！, @end@]
PRED: ['这', '是', '我', '的', '？']
SOURCE: [@start@, It, does, n't, surprise, me, ., @end@]
GOLD: [@start@, 这, 并, 不, 让, 我, 惊, 讶, 。, @end@]
PRED: ['这', '是', '我', '的', '。']
SOURCE: [@start@, I, 'm, not, a, real, fish, ,, I, 'm, just, a, mere, plushy, ., @end@]
GOLD: [@start@, 我, 不, 是

unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/allennlp/common/util.py", line 378, in gpu_memory_mb
    encoding='utf-8')
  File "/usr/lib/python3.6/subprocess.py", line 356, in check_output
    **kwargs).stdout
  File "/usr/lib/python3.6/subprocess.py", line 438, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.

  0%|          | 0/1169 [00:00<?, ?it/s]

...


loss: 4.2639 ||:   0%|          | 1/1169 [00:01<25:47,  1.33s/it]
loss: 3.8729 ||:   0%|          | 2/1169 [00:01<19:39,  1.01s/it]
loss: 3.7868 ||:   0%|          | 3/1169 [00:01<15:19,  1.27it/s]
loss: 3.6950 ||:   0%|          | 4/1169 [00:02<11:52,  1.63it/s]
loss: 3.6811 ||:   0%|          | 5/1169 [00:02<09:53,  1.96it/s]
loss: 3.7399 ||:   1%|          | 6/1169 [00:02<08:50,  2.19it/s]
loss: 3.7316 ||:   1%|          | 7/1169 [00:02<07:46,  2.49it/s]
loss: 3.7800 ||:   1%|          | 8/1169 [00:03<07:25,  2.61it/s]
loss: 3.7852 ||:   1%|          | 9/1169 [00:03<06:53,  2.80it/s]
loss: 3.7717 ||:   1%|          | 10/1169 [00:03<06:47,  2.84it/s]
loss: 3.7716 ||:   1%|          | 11/1169 [00:04<06:35,  2.93it/s]
loss: 3.7573 ||:   1%|          | 12/1169 [00:04<05:49,  3.31it/s]
loss: 3.7345 ||:   1%|          | 13/1169 [00:04<05:16,  3.65it/s]
loss: 3.7305 ||:   1%|          | 14/1169 [00:04<05:26,  3.54it/s]
loss: 3.7730 ||:   1%|▏         | 15/1169 [00:05<06:22,  3.02it/s]
los

SOURCE: [@start@, I, have, to, go, to, sleep, ., @end@]
GOLD: [@start@, 我, 该, 去, 睡, 觉, 了, 。, @end@]
PRED: ['我', '昨', '天', '早', '上', '回', '家', '。']
SOURCE: [@start@, I, just, do, n't, know, what, to, say, ., @end@]
GOLD: [@start@, 我, 就, 是, 不, 知, 道, 說, 些, 什, 麼, 。, @end@]
PRED: ['我', '不', '知', '道', '汤', '姆', '说', '话', '。']
SOURCE: [@start@, I, may, give, up, soon, and, just, nap, instead, ., @end@]
GOLD: [@start@, 也, 许, 我, 会, 马, 上, 放, 弃, 然, 后, 去, 睡, 一, 觉, 。, @end@]
PRED: ['我', '知', '道', '我', '們', '在', '這', '裡', '的', '时', '候', '到', '了', '。']
SOURCE: [@start@, I, 'm, going, to, go, ., @end@]
GOLD: [@start@, 我, 要, 走, 了, 。, @end@]
PRED: ['我', '想', '吃', '。']
SOURCE: [@start@, That, 's, MY, line, !, @end@]
GOLD: [@start@, 那, 是, 我, 的, 台, 词, ！, @end@]
PRED: ['这', '是', '我', '的', '！']
SOURCE: [@start@, It, does, n't, surprise, me, ., @end@]
GOLD: [@start@, 这, 并, 不, 让, 我, 惊, 讶, 。, @end@]
PRED: ['这', '是', '我', '的', '。']
SOURCE: [@start@, I, 'm, not, a, real, fish, ,, I, 'm, just, a, mere, plushy, ., 

unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/allennlp/common/util.py", line 378, in gpu_memory_mb
    encoding='utf-8')
  File "/usr/lib/python3.6/subprocess.py", line 356, in check_output
    **kwargs).stdout
  File "/usr/lib/python3.6/subprocess.py", line 438, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.

  0%|          | 0/1169 [00:00<?, ?it/s]

PRED: ['这', '是', '我', '的', '。']
SOURCE: [@start@, Thanks, for, having, explained, to, me, at, last, why, people, take, me, for, an, idiot, ., @end@]
GOLD: [@start@, 感, 谢, 最, 后, 为, 我, 说, 明, 了, 为, 什, 么, 人, 们, 把, 我, 当, 作, 傻, 瓜, 了, 。, @end@]
PRED: ['我', '们', '不', '知', '道', '我', '们', '说', '话', '，', '我', '们', '说', '话', '。']
Epoch: 2



loss: 3.5288 ||:   0%|          | 1/1169 [00:01<24:30,  1.26s/it]
loss: 3.4158 ||:   0%|          | 2/1169 [00:01<19:08,  1.02it/s]
loss: 3.5326 ||:   0%|          | 3/1169 [00:02<16:12,  1.20it/s]
loss: 3.6278 ||:   0%|          | 4/1169 [00:02<14:06,  1.38it/s]
loss: 3.5472 ||:   0%|          | 5/1169 [00:02<11:57,  1.62it/s]
loss: 3.5314 ||:   1%|          | 6/1169 [00:03<10:24,  1.86it/s]
loss: 3.6612 ||:   1%|          | 7/1169 [00:03<10:47,  1.80it/s]
loss: 3.5875 ||:   1%|          | 8/1169 [00:04<08:51,  2.19it/s]
loss: 3.5096 ||:   1%|          | 9/1169 [00:04<07:40,  2.52it/s]
loss: 3.4691 ||:   1%|          | 10/1169 [00:04<06:52,  2.81it/s]
loss: 3.4926 ||:   1%|          | 11/1169 [00:05<07:05,  2.72it/s]
loss: 3.5099 ||:   1%|          | 12/1169 [00:05<07:16,  2.65it/s]
loss: 3.5367 ||:   1%|          | 13/1169 [00:05<07:31,  2.56it/s]
loss: 3.5228 ||:   1%|          | 14/1169 [00:06<07:14,  2.66it/s]
loss: 3.5443 ||:   1%|▏         | 15/1169 [00:06<07:33,  2.54it/s]
los